In [1]:
import pandas as pd

In [2]:
weight = [63, 21, 2, 32, 13, 80, 19, 37, 56, 41, 14, 8, 32, 42, 7]
value = [13, 2, 20, 10, 7, 14, 7, 2, 2, 4, 16, 17, 17, 3, 21]
objects = pd.DataFrame({'weights': weight, 'values': value})

In [3]:
class backpack:
  def __init__(self, array, tabu_wait):
    self.inside = array
    self.total_weight = sum(objects.iloc[array, 0])
    self.limit_total_weight = 275
    self.total_value = sum(objects.iloc[array, 1])
    self.tabu_list = pd.DataFrame(columns=['array', 'waitlist'])
    self.tabu_wait = tabu_wait
  
  def calculate_tabu_list(self):

    #Sub 1 in tabu_list
    sub_waitlist = self.tabu_list['waitlist'] - 1
    self.tabu_list['waitlist'] = sub_waitlist
    self.delete_zeros()


    self.tabu_list = self.tabu_list.append({'array': self.inside,
                           'waitlist':self.tabu_wait}, ignore_index=True)

    list_selecteds = pd.DataFrame(columns = ['array',
                                             'weight_difference',
                                             'value_difference'])
    
    actual_array = set(self.inside)
    entire_array = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14}


    #Verify if we can add some object without the swap need
    if self.total_weight + objects.iloc[list(entire_array - actual_array), 0].min() <= self.limit_total_weight:
      possible_adds = self.get_bests_values_to_add(actual_array, entire_array, list_selecteds)
      list_selecteds = list_selecteds.append(possible_adds, ignore_index=True)
    
    list_selecteds = self.swap_possibles(actual_array, entire_array, list_selecteds)
    return list_selecteds

  def swap_possibles(self, actual_array, entire_array, df):
    rest_of_values = list(entire_array - actual_array)
    for value in rest_of_values:
      df = self.one_swap_possible(value, self.inside, df)
    return df

  def one_swap_possible(self, objectID, array, df):
    for i in range(len(array)):
      array_copy = array.copy()
      array_copy[i] = objectID

      sum_weight = sum(objects.iloc[array_copy, 0])
      if sum_weight <= self.limit_total_weight:
        sum_value = sum(objects.iloc[array_copy, 1])
        array_copy.sort()
        df = df.append({'array': array_copy,
                      'weight_difference': sum_weight - self.total_weight,
                      'value_difference': sum_value - self.total_value}, ignore_index=True)
      
    return df

  def get_bests_values_to_add(self, actual_array, entire_array, df):
    rest_of_values = objects.iloc[list(entire_array - actual_array)]
    possible_values = rest_of_values[rest_of_values['weights'] + self.total_weight <= self.limit_total_weight]
    for index in possible_values.index:
      possible_array = list(actual_array).copy()
      possible_array.append(index)
      new_total_weight = sum(objects.iloc[possible_array, 0])
      new_total_value = sum(objects.iloc[possible_array, 1])

      df = df.append({'array': possible_array,
                      'weight_difference': new_total_weight - self.total_weight,
                      'value_difference': new_total_value - self.total_value}, ignore_index=True)
      
    return df
  
  def change_array(self, array):
    array.sort()
    self.inside = array
    self.total_weight = sum(objects.iloc[array, 0])
    self.total_value = sum(objects.iloc[array, 1])
    

    return self

  def delete_zeros(self):
    indexNames = self.tabu_list[self.tabu_list['waitlist'] == 0 ].index
    self.tabu_list.drop(indexNames , inplace=True)




In [4]:
def organize_possible_selecteds_list(df):
  df.sort_values(by=['value_difference'], inplace=True, ascending=False)

In [5]:
def check_tabu_list(df, array):
  for row in df['array']:
    array.sort()
    row.sort()
    if array == row:
      return True
  return False

In [6]:
#Tabu Search

actual_solution = backpack([0, 1, 2, 3, 6, 10, 11, 12], 10)
iters = 100

best_solution = actual_solution.inside.copy()
best_value = actual_solution.total_value
i = 0

while i < iters:

  possibles = actual_solution.calculate_tabu_list()
  organize_possible_selecteds_list(possibles)
  for array in possibles['array']:
    if check_tabu_list(actual_solution.tabu_list, array):
      #Aspiration_criteria
      if best_value < sum(objects.iloc[array, 1]):
        actual_solution.change_array(array)
        best_value = actual_solution.total_value
        best_solution = array.copy()
        break
      else:
        pass
    else:
      actual_solution.change_array(array)
      if best_value < actual_solution.total_value:
        best_value = actual_solution.total_value
        best_solution = actual_solution.inside.copy()
      break
  

  i+=1

print(f'Best Value is {best_value}')
new_list = [x+1 for x in best_solution]
print(f'Items in the backpack {new_list}')
print(f'Total Weight {sum(objects.iloc[best_solution, 0])}')

Best Value is 142
Items in the backpack [1, 3, 4, 5, 6, 7, 11, 12, 13, 15]
Total Weight 270


###Busca Tabu
O algoritmo de Busca Tabu se mostrou eficiente para o problema, com baixo consumo de memória, pois o que define o tamanho da memória utilizada é um parâmetro escolhido pelo usuário.


Quanto ao processamento do algoritmo, demonstrou ser rapido para resolver o problema da mochila, seguindo o numero de iterações que também é fornecido pelo usuário.


Para problemas como o da mochila, onde não nos interessa o caminho realizado pelo algoritmo e sim apenas a resposta, é mais interessante de se utilizar do que A* e os demais algoritmos vistos anteriormente, pelo fato do consumo da memória ser muito menor e termos um controle maior do tempo de execução.Porém, não temos garantia de achar exatamente a melhor solução.